In [2]:
# Library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_profiling as pdp
import os
%matplotlib inline
import seaborn as sns
from tqdm.auto import tqdm
import statsmodels
import statsmodels.api as sm

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_pinball_loss
from lightgbm import LGBMRegressor

import warnings
warnings.simplefilter('ignore')

In [3]:
# pandas setting
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [4]:
df = pd.read_csv('../data/train_data.csv')

In [5]:
df.tail()

,id,date,highest,lowest,rain,ice1,ice2,ice3,oden1,oden2,oden3,oden4,hot1,hot2,hot3,dessert1,dessert2,dessert3,dessert4,dessert5,drink1,drink2,drink3,drink4,drink5,drink6,alcol1,alcol2,alcol3,snack1,snack2,snack3,bento1,bento2,bento3,bento4,tild1,tild2,men1,men2,men3,men4,men5,men6
345,346,3/22,23.9,8.7,0.0,27,77,30,29,36,77,53,147,197,206,24,44,6,26,32,65,53,30,50,25,8,73,82,82,29,24,39,10,15,17,9,5,8,23,12,13,35,4,17
346,347,3/23,9.0,4.4,0.0,14,33,32,76,73,159,96,116,221,207,24,38,5,26,37,17,20,16,31,24,11,93,99,93,19,11,39,50,37,72,26,5,8,51,31,39,40,37,39
347,348,3/24,15.4,1.1,0.0,20,51,34,104,108,185,134,105,193,189,45,65,50,59,61,19,25,14,33,23,8,101,104,109,23,21,45,48,46,70,28,5,10,51,31,39,37,31,38
348,349,3/25,15.0,3.9,0.0,19,50,33,79,91,160,94,111,211,207,20,38,5,29,38,20,21,14,27,26,9,43,20,38,33,24,42,8,11,20,8,6,8,21,9,11,27,4,18
349,350,3/26,15.9,9.7,0.0,19,53,31,18,30,69,42,161,232,251,34,61,15,41,55,24,25,15,30,26,9,51,31,50,27,10,28,46,40,63,27,6,7,52,30,36,41,47,36


In [8]:
# Data
train = pd.read_csv('../data/train_data.csv')
test = pd.read_csv('../data/test_data.csv')
submission = pd.read_csv('../data/submission.csv')

In [9]:
# Target Columns
target_columns = ['ice1', 'ice2', 'ice3', 'oden1', 'oden2', 'oden3', 'oden4', 'hot1',
       'hot2', 'hot3', 'dessert1', 'dessert2', 'dessert3', 'dessert4',
       'dessert5', 'drink1', 'drink2', 'drink3', 'drink4', 'drink5', 'drink6',
       'alcol1', 'alcol2', 'alcol3', 'snack1', 'snack2', 'snack3', 'bento1',
       'bento2', 'bento3', 'bento4', 'tild1', 'tild2', 'men1', 'men2', 'men3',
       'men4', 'men5', 'men6']

In [17]:
# 日付をdatetimeに変更する関数
def change_to_datetime(df, mode='train'):
    df_tmp = df.copy()
    this_year = 2021 # 2022のテストデータを予測すると仮定
    
    df_tmp['date'] = pd.to_datetime(df_tmp.date, format='%m/%d')
    df_tmp['year'] = df_tmp['date'].dt.year
    df_tmp['month'] = df_tmp['date'].dt.month
    df_tmp['day'] = df_tmp['date'].dt.day
    
    if mode=='train':
        df_tmp.loc[df_tmp['month']>3, 'year'] = this_year
        df_tmp.loc[df_tmp['month']<=3, 'year'] = this_year + 1
    else:
        df_tmp['year'] = this_year + 1
    
    df_tmp['date'] = pd.to_datetime({'year': df_tmp.year, 'month': df_tmp.month, 'day': df_tmp.day})
    df_tmp['weekday'] = df_tmp['date'].dt.weekday # 月=0, 日=6
    return df_tmp

In [29]:
# # 日付をdatetimeに変更する
train_df = change_to_datetime(train, mode='train')
test_df = change_to_datetime(test, mode='test')
all_df = pd.concat([train_df, test_df]).reset_index(drop=True)

In [32]:
# カラム入れ替え
train_df = train_df.reindex(columns=['id', 'date', 'year', 'month', 'day', 'weekday', 'highest', 'lowest', 'rain', 'ice1', 'ice2', 'ice3',
       'oden1', 'oden2', 'oden3', 'oden4', 'hot1', 'hot2', 'hot3', 'dessert1',
       'dessert2', 'dessert3', 'dessert4', 'dessert5', 'drink1', 'drink2',
       'drink3', 'drink4', 'drink5', 'drink6', 'alcol1', 'alcol2', 'alcol3',
       'snack1', 'snack2', 'snack3', 'bento1', 'bento2', 'bento3', 'bento4',
       'tild1', 'tild2', 'men1', 'men2', 'men3', 'men4', 'men5', 'men6'
       ])
test_df = test_df.reindex(columns=['id', 'date', 'year', 'month', 'day', 'weekday', 'highest', 'lowest', 'rain'])

In [33]:
train_df.head()

,id,date,year,month,day,weekday,highest,lowest,rain,ice1,ice2,ice3,oden1,oden2,oden3,oden4,hot1,hot2,hot3,dessert1,dessert2,dessert3,dessert4,dessert5,drink1,drink2,drink3,drink4,drink5,drink6,alcol1,alcol2,alcol3,snack1,snack2,snack3,bento1,bento2,bento3,bento4,tild1,tild2,men1,men2,men3,men4,men5,men6
0,1,2021-04-11,2021,4,11,6,21.9,12.4,0.0,25,72,26,10,23,52,35,180,254,270,42,58,50,59,67,54,45,28,49,22,8,63,51,59,26,21,35,56,46,70,27,12,12,57,30,41,38,37,35
1,2,2021-04-12,2021,4,12,0,25.9,13.9,0.0,30,85,33,9,18,42,26,202,219,235,22,36,5,28,37,69,54,35,58,22,9,77,66,72,36,32,63,8,14,23,9,5,8,19,9,13,26,4,16
2,3,2021-04-13,2021,4,13,1,20.9,11.9,0.0,21,68,28,12,22,57,31,164,210,223,20,41,5,30,32,46,38,24,45,26,9,81,69,74,36,25,57,9,12,19,6,4,9,23,9,11,33,4,13
3,4,2021-04-14,2021,4,14,2,18.8,11.4,0.0,19,62,35,13,29,62,33,193,242,251,21,43,5,28,32,35,31,22,37,23,8,88,92,81,20,10,45,54,43,74,28,15,17,55,35,46,46,51,46
4,5,2021-04-15,2021,4,15,3,22.1,13.5,19.0,21,72,32,10,24,44,33,218,271,274,44,63,47,59,63,51,39,20,52,28,14,116,135,119,31,21,39,52,36,69,26,4,9,54,33,39,40,41,40


In [34]:
test_df.head()

,id,date,year,month,day,weekday,highest,lowest,rain
0,1,2022-03-27,2022,3,27,6,19.7,7.3,0.0
1,2,2022-03-28,2022,3,28,0,16.9,9.0,0.0
2,3,2022-03-29,2022,3,29,1,9.3,6.8,0.0
3,4,2022-03-30,2022,3,30,2,11.7,7.2,3.5
4,5,2022-03-31,2022,3,31,3,16.3,7.3,1.5


In [41]:
# csvファイルへ保存
# 2021からのデータと考えた場合
train_df.to_csv('../work/train_preprocessed_from2021.csv', index=False)
test_df.to_csv('../work/test_preprocessed_from2021.csv', index=False)